# Crawling Berita

In [7]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import time
import re
import concurrent.futures
from urllib.parse import urljoin

# --- PENGATURAN ---
MAX_WORKERS = 10
MAX_ARTICLES_PER_SUBCATEGORY = 25
BASE_URL = "https://www.cnnindonesia.com"

def get_category_links():
    """Tahap 1: Mengambil semua link kategori dan sub-kategori dari navigasi utama."""
    print("Mencari daftar kategori dan sub-kategori...")
    categories = []
    try:
        response = requests.get(BASE_URL, timeout=10)
        response.raise_for_status()
        soup = BeautifulSoup(response.content, "html.parser")
        
        nav_items = soup.select("nav > ul > li.header-nhl")
        if not nav_items:
            nav_items = soup.select("nav > ul > li")

        for nav_item in nav_items:
            main_category_tag = nav_item.find('a', recursive=False)
            if not main_category_tag or not main_category_tag.has_attr('href'):
                continue
            
            main_category_name = main_category_tag.text.strip()
            
            sub_menu_div = nav_item.find('div', class_='navbar__item-dropdown')
            if sub_menu_div:
                sub_category_links = sub_menu_div.find_all('a')
                for sub_link in sub_category_links:
                    if sub_link.has_attr('href'):
                        categories.append({
                            'kategori': main_category_name,
                            'sub_kategori': sub_link.text.strip(),
                            'url': urljoin(BASE_URL, sub_link['href'])
                        })
    except requests.exceptions.RequestException as e:
        print(f"Gagal mengambil kategori: {e}")
    
    unique_categories = [dict(t) for t in {tuple(d.items()) for d in categories if d['sub_kategori']}]
    print(f"Ditemukan {len(unique_categories)} sub-kategori unik.\n")
    return unique_categories

def get_article_info_from_subcategory(subcategory):
    """
    Tahap 2: Mengambil info artikel (URL + Kategori) dari satu halaman sub-kategori.
    MODIFIED: Now returns a list of dictionaries, not just URLs.
    """
    articles_to_scrape = []
    page = 1
    print(f"--> Mengambil artikel dari '{subcategory['kategori']} - {subcategory['sub_kategori']}'...")
    while len(articles_to_scrape) < MAX_ARTICLES_PER_SUBCATEGORY:
        try:
            paginated_url = f"{subcategory['url']}/page/{page}"
            response = requests.get(paginated_url, timeout=10)
            if response.status_code != 200: break
            
            soup = BeautifulSoup(response.content, "html.parser")
            articles_on_page = soup.select("article a")
            
            if not articles_on_page: break
            
            for link in articles_on_page:
                if link.has_attr('href'):
                    # Simpan URL bersama dengan info kategori dan sub-kategorinya
                    articles_to_scrape.append({
                        'url': link['href'],
                        'kategori': subcategory['kategori'],
                        'sub_kategori': subcategory['sub_kategori']
                    })
                    if len(articles_to_scrape) >= MAX_ARTICLES_PER_SUBCATEGORY:
                        break
            page += 1
            time.sleep(0.2)
        except requests.exceptions.RequestException:
            break
    return articles_to_scrape

def scrape_article_detail(article_info):
    """
    Tahap 3: Mengekstrak detail dari satu artikel.
    MODIFIED: Now accepts a dictionary 'article_info' to access the passed-down category names.
    """
    article_url = article_info['url']
    try:
        response = requests.get(article_url, timeout=15)
        response.raise_for_status()
        soup = BeautifulSoup(response.content, "html.parser")

        id_match = re.search(r'-(\d+)$', article_url.split('/')[-1])
        id_berita = id_match.group(1) if id_match else None

        judul_berita = soup.select_one("h1.text-xl").text.strip()

        content_element = soup.select_one("div.detail-text")
        isi_berita = "Isi berita tidak ditemukan"
        if content_element:
            for unwanted in content_element.select("div, table, blockquote"):
                unwanted.decompose()
            isi_berita_mentah = content_element.get_text(separator=" ", strip=True)
            isi_berita = re.sub(r'\s+', ' ', isi_berita_mentah).strip()

        # Gabungkan data yang di-scrape dengan data kategori yang sudah ada
        return {
            "id_berita": id_berita,
            "kategori": article_info['kategori'],
            "sub_kategori": article_info['sub_kategori'],
            "judul_berita": judul_berita,
            "isi_berita": isi_berita,
        }
    except Exception:
        return None

def main():
    """Fungsi utama untuk mengorkestrasi semua tahapan scraping."""
    start_time = time.time()
    
    subcategories = get_category_links()
    if not subcategories: return
    
    all_articles_to_scrape = []
    for subcat in subcategories:
        # Nama fungsi diubah agar lebih jelas
        info_list = get_article_info_from_subcategory(subcat)
        all_articles_to_scrape.extend(info_list)
        print(f"    Terkumpul {len(info_list)} artikel.")
    
    print(f"\nTotal artikel yang akan di-scrape: {len(all_articles_to_scrape)}\n")
    
    final_results = []
    with concurrent.futures.ThreadPoolExecutor(max_workers=MAX_WORKERS) as executor:
        # Kirim list of dictionaries ke fungsi scrape_article_detail
        results = executor.map(scrape_article_detail, all_articles_to_scrape)
        
        for i, result in enumerate(results):
            if result:
                final_results.append(result)
                print(f"✔️ Selesai: Artikel ke-{i+1} | {result['judul_berita'][:50]}...")

    df = pd.DataFrame(final_results)
    if not df.empty:
        # Tambahkan kolom baru ke urutan
        df = df[['id_berita', 'kategori', 'sub_kategori', 'judul_berita', 'isi_berita']]
    
    output_filename = "berita_cnn_indonesia_lengkap.csv"
    df.to_csv(output_filename, index=False)
    
    end_time = time.time()
    print("\n\n✅ Proses scraping selesai.")
    print(f"Total data yang berhasil diambil: {len(df)} baris.")
    print(f"File disimpan sebagai: {output_filename}")
    print(f"Total waktu eksekusi: {end_time - start_time:.2f} detik.")
    
    return df

if __name__ == "__main__":
    df_hasil_cnn = main()
    if df_hasil_cnn is not None and not df_hasil_cnn.empty:
        print("\nPratinjau Hasil Data:")
        print(df_hasil_cnn.head())

Mencari daftar kategori dan sub-kategori...
Ditemukan 0 sub-kategori unik.



In [8]:
# Impor library yang dibutuhkan
import pandas as pd
import requests
from bs4 import BeautifulSoup
import time
import re
import concurrent.futures
from urllib.parse import urljoin

# --- PENGATURAN GLOBAL ---
# Aturan ini bisa diubah sesuai kebutuhan
MAX_WORKERS = 10  # Jumlah "pekerja" paralel untuk mempercepat proses
MAX_ARTICLES_PER_SUBCATEGORY = 50 # Batas artikel per sub-kategori agar tidak terlalu lama
BASE_URL = "https://www.cnnindonesia.com"
HEADERS = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36"}

def get_category_links():
    """TAHAP 1: DISCOVERY
    Menemukan semua 'alamat' sub-kategori dari menu navigasi utama.
    Ini adalah langkah perencanaan untuk mengetahui area mana saja yang akan di-scrape."""
    print("Mencari daftar kategori dan sub-kategori...")
    categories = []
    try:
        response = requests.get(BASE_URL, headers=HEADERS, timeout=10)
        response.raise_for_status()
        soup = BeautifulSoup(response.content, "html.parser")
        
        # Selektor untuk menemukan setiap item di menu navigasi
        nav_items = soup.select("nav > ul > li.header-nhl_dropdown-item")
        
        for nav_item in nav_items:
            main_category_tag = nav_item.find('a', recursive=False)
            if not main_category_tag: continue
            
            main_category_name = main_category_tag.text.strip()
            
            # Selektor untuk menemukan 'kotak' dropdown berisi sub-kategori
            sub_menu_div = nav_item.find('div', class_='navbar_item-dropdown')
            if sub_menu_div:
                for sub_link in sub_menu_div.find_all('a'):
                    if sub_link.has_attr('href'):
                        categories.append({
                            'kategori': main_category_name,
                            'sub_kategori': sub_link.text.strip(),
                            'url': urljoin(BASE_URL, sub_link['href'])
                        })
    except Exception as e:
        print(f"Gagal mengambil kategori: {e}")
    
    unique_categories = [dict(t) for t in {tuple(d.items()) for d in categories if d['sub_kategori']}]
    print(f"Ditemukan {len(unique_categories)} sub-kategori unik.\n")
    return unique_categories

def get_article_info_from_subcategory(subcategory):
    """TAHAP 2: COLLECTION
    Mengunjungi setiap 'alamat' sub-kategori dan mengumpulkan semua link artikel di dalamnya,
    lengkap dengan info kategori agar tidak hilang."""
    articles_to_scrape = []
    page = 1
    print(f"--> Mengambil artikel dari '{subcategory['kategori']} - {subcategory['sub_kategori']}'...")
    while len(articles_to_scrape) < MAX_ARTICLES_PER_SUBCATEGORY:
        try:
            paginated_url = f"{subcategory['url']}/page/{page}"
            response = requests.get(paginated_url, headers=HEADERS, timeout=10)
            if response.status_code != 200: break
            
            soup = BeautifulSoup(response.content, "html.parser")
            # Selektor untuk menemukan link artikel di halaman kategori
            articles_on_page = soup.select("article a")
            
            if not articles_on_page: break
            
            for link in articles_on_page:
                if link.has_attr('href'):
                    articles_to_scrape.append({
                        'url': link['href'],
                        'kategori': subcategory['kategori'],
                        'sub_kategori': subcategory['sub_kategori']
                    })
                    if len(articles_to_scrape) >= MAX_ARTICLES_PER_SUBCATEGORY: break
            page += 1
            time.sleep(0.2)
        except Exception:
            break
    return articles_to_scrape

def scrape_article_detail(article_info):
    """TAHAP 3: EXTRACTION
    Mengunjungi satu per satu link artikel dan 'mencomot' data spesifik
    seperti ID, judul (yang akurat), dan isi beritanya."""
    article_url = article_info['url']
    try:
        response = requests.get(article_url, headers=HEADERS, timeout=15)
        soup = BeautifulSoup(response.content, "html.parser")

        # Ekstrak ID unik dari URL
        id_match = re.search(r'-(\d+)$', article_url.split('/')[-1])
        id_berita = id_match.group(1) if id_match else None

        # Ekstrak judul dari tag H1 untuk akurasi maksimal
        judul_berita = soup.select_one("h1.text-xl").text.strip()

        # Ekstrak semua paragraf dari container isi berita
        content_element = soup.select_one("div.detail-text")
        isi_berita = "Isi berita tidak ditemukan"
        if content_element:
            paragraf = [p.get_text(strip=True) for p in content_element.select("p")]
            isi_berita = " ".join(paragraf)

        return {
            "id_berita": id_berita,
            "kategori": article_info['kategori'],
            "sub_kategori": article_info['sub_kategori'],
            "judul_berita": judul_berita,
            "isi_berita": isi_berita,
            "link": article_url
        }
    except Exception:
        return None

def main():
    """FUNGSI UTAMA: ORKESTRATOR
    Mengatur alur kerja dari awal hingga akhir, dari perencanaan hingga penyimpanan hasil."""
    start_time = time.time()
    
    # Menjalankan Tahap 1
    subcategories = get_category_links()
    if not subcategories: return
    
    # Menjalankan Tahap 2
    all_articles_to_scrape = []
    for subcat in subcategories:
        info_list = get_article_info_from_subcategory(subcat)
        all_articles_to_scrape.extend(info_list)
        print(f"    Terkumpul {len(info_list)} artikel.")
    
    print(f"\nTotal artikel yang akan di-scrape: {len(all_articles_to_scrape)}\n")
    
    # Menjalankan Tahap 3 secara paralel untuk kecepatan
    final_results = []
    with concurrent.futures.ThreadPoolExecutor(max_workers=MAX_WORKERS) as executor:
        results = executor.map(scrape_article_detail, all_articles_to_scrape)
        
        for i, result in enumerate(results):
            if result and result['isi_berita'] and len(result['isi_berita']) > 50: # Filter berita tanpa isi
                final_results.append(result)
                print(f"✔️ Selesai: Artikel ke-{i+1} | {result['judul_berita'][:50]}...")

    # Tahap Akhir: Menyimpan hasil ke dalam file
    df = pd.DataFrame(final_results)
    if not df.empty:
        df = df[['id_berita', 'kategori', 'sub_kategori', 'judul_berita', 'isi_berita', 'link']]
    
    output_filename = "berita_cnn_indonesia_final.csv"
    df.to_csv(output_filename, index=False, encoding='utf-8-sig')
    
    end_time = time.time()
    print("\n\n✅ Proses scraping selesai.")
    print(f"Total data yang berhasil diambil: {len(df)} baris.")
    print(f"File disimpan sebagai: {output_filename}")
    print(f"Total waktu eksekusi: {end_time - start_time:.2f} detik.")
    
    return df

# Titik awal program dijalankan
if __name__ == "__main__":
    df_hasil_cnn = main()
    if df_hasil_cnn is not None and not df_hasil_cnn.empty:
        print("\nPratinjau Hasil Data:")
        # Menampilkan hasil dengan lebar kolom lebih besar agar mudah dibaca
        pd.set_option('display.max_colwidth', 80)
        print(df_hasil_cnn.head())

Mencari daftar kategori dan sub-kategori...
Ditemukan 0 sub-kategori unik.



In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import uuid # Meskipun tidak digunakan untuk ID dari web, bisa untuk ID unik jika diperlukan

In [14]:
# URL homepage CNN Indonesia
url = "https://www.cnnindonesia.com/"
headers = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/140.0.0.0 Safari/537.36 Edg/140.0.0.0"}

# Ambil halaman utama
response = requests.get(url, headers=headers)
soup = BeautifulSoup(response.text, "html.parser")

data = []

# Loop semua link artikel di homepage
for artikel in soup.select("article a"):
    link = artikel.get("href")
    judul = artikel.get_text(strip=True)

    # Skip jika link kosong atau bukan http
    if not link or not link.startswith("http"):
        continue

    try:
        # Ambil halaman detail berita
        resp_detail = requests.get(link, headers=headers)
        soup_detail = BeautifulSoup(resp_detail.text, "html.parser")

        # --- PERUBAHAN DIMULAI DI SINI ---
        
        # Ambil ID Berita dari link
        id_berita = None
        try:
            # Contoh path: /ekonomi/20250910153012-92-1234567/nama-artikel
            # Kita ambil bagian sebelum nama artikel, yaitu '20250910153012-92-1234567'
            id_segment = link.split("/")[-2]
            # Kemudian kita pisah dengan '-' dan ambil bagian terakhirnya
            id_berita = id_segment.split("-")[-1]
        except (IndexError, AttributeError):
            # Jika struktur URL berbeda dan gagal, ID akan tetap None
            id_berita = None

        # Ambil kategori dari link (path pertama setelah domain)
        kategori = None
        try:
            path = link.replace("https://www.cnnindonesia.com/", "")
            kategori = path.split("/")[0] if path else None
        except Exception:
            kategori = None

        # Ambil isi berita
        paragraf = [p.get_text(strip=True) for p in soup_detail.select("div.detail-text p")]
        isi = " ".join(paragraf)

        if isi:  # hanya simpan kalau ada isi berita
            data.append({
                "id_berita": id_berita, # <-- Kolom baru ditambahkan di sini
                "judul": judul,
                "kategori": kategori,
                "isi": isi,
                "link": link
            })
            
        # --- PERUBAHAN SELESAI DI SINI ---

    except Exception as e:
        print(f"Gagal ambil {link}: {e}")

# Simpan ke dataframe Pandas
df = pd.DataFrame(data)

# Simpan ke file CSV
df.to_csv("berita_cnn_dengan_id.csv", index=False, encoding="utf-8-sig")

# Tampilkan hasil di notebook
pd.set_option("display.max_colwidth", 100)  # biar isi tidak kepotong
print("Proses scraping selesai. Berikut adalah contoh datanya:")
display(df)

Proses scraping selesai. Berikut adalah contoh datanya:


,id_berita,judul,kategori,isi,link
0,1271886,Komisi III DPR Mulai Rapat Maraton Seleksi Calon Hakim Agung,nasional,Komisi III DPR RI mulai menggelar rapat maraton untuk melakukan seleksi atau uji kelayakan dan k...,https://www.cnnindonesia.com/nasional/20250910041637-32-1271886/komisi-iii-dpr-mulai-rapat-marat...
1,1271880,Rieke PDIP Usul Prabowo Terbitkan Perpres Jaminan Sosial buat Ojol,nasional,"Anggota Komisi VI DPR dari FraksiPDIP, Rieke Diah Pitaloka alias Oneng meminta Presiden Prabowo ...",https://www.cnnindonesia.com/nasional/20250910030753-32-1271880/rieke-pdip-usul-prabowo-terbitka...
2,1272108,"01VIDEO: Panggung Perdana iPhone Air, Hp Tertipis Buatan AppleTeknologi",teknologi,"Apple merilis iPhone Air pada acara peluncuran produk tahunan, Selasa (9/9). iPhone dengan model...",https://www.cnnindonesia.com/teknologi/20250910131437-210-1272108/video-panggung-perdana-iphone-...
3,1272101,"02VIDEO: Banjir Terjang Bali, Dua Orang Meninggal DuniaNasional",nasional,Banjir besar melanda Bali akibat hujan lebat sejak Selasa (9/9) hingga Rabu (10/9). Salah satu w...,https://www.cnnindonesia.com/nasional/20250910125046-24-1272101/video-banjir-terjang-bali-dua-or...
4,1272087,03VIDEO: Kata-kata Qatar usai Dihantam Serangan Udara IsraelInternasional,internasional,Pemerintah Qatar mengecam serangan Israel terhadap para pemimpin Hamas di Doha pada Selasa (9/9)...,https://www.cnnindonesia.com/internasional/20250910120421-124-1272087/video-kata-kata-qatar-usai...
5,1272073,04VIDEO: Momen Yusril Temui Delpedro di Sel TahananNasional,nasional,Direktur Lokataru Foundation Delpedro Marhaen menyatakan tidak bersalah dalam kasus dugaan pengh...,https://www.cnnindonesia.com/nasional/20250910105042-16-1272073/video-momen-yusril-temui-delpedr...
6,1272137,05VIDEO: Detik-detik Menkeu Nepal Dikejar Massa hingga Terjun ke SungaiInternasional,internasional,Sebuah rekaman video menunjukkan Menteri Keuangan Nepal Bishnu Prasad Paudel dikejar dan dihajar...,https://www.cnnindonesia.com/internasional/20250910135637-117-1272137/video-detik-detik-menkeu-n...
7,1272165,"06Ada Putusan MK, TNI Masih Pertimbangkan Polisikan Ferry IrwandiNasional",nasional,TNImasih mempertimbangkan langkah hukum yang akan diambil terhadap CEO Malaka ProjectFerry Irwan...,https://www.cnnindonesia.com/nasional/20250910145755-12-1272165/ada-putusan-mk-tni-masih-pertimb...
8,1271984,"Bali Diterjang Banjir Besar, Dua Warga Meninggal Dunia",nasional,"Banjirbesar melandaBali, menerjang sejumlah daerah seperti Kabupaten Jembrana dan Kota Denpasar ...",https://www.cnnindonesia.com/nasional/20250910093338-20-1271984/bali-diterjang-banjir-besar-dua-...
9,1271909,"Timnas Indonesia era Kluivert: Inverted Fullback, Ball Winner, Mezzala",olahraga,Tak berlebihan kiranya jikaPatrick Kluivertdisebut mulai memperlihatkan jati dirinya diTimnas In...,https://www.cnnindonesia.com/olahraga/20250910061945-142-1271909/timnas-indonesia-era-kluivert-i...
